In [1]:
import ROOT
import uproot
import numpy as np
import array as array
import math
import matplotlib.pyplot as plt

#import statistics as st
ROOT.gStyle.SetOptStat(000000)

Welcome to JupyROOT 6.26/04


In [2]:
import subprocess
import sys
from unfold_utils import binning
def install_package(package_name):
    # Install the package using subprocess to call pip
    subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

package_name = "pickle5"  # Replace with the package you want to install

try:
    import requests  # Replace with the module name of the package
except ImportError:
    print(f"{package_name} not found. Installing...")
    install_package(package_name)
    import requests  # Import again after installation

def import_packages():
    try:
        import mplhep  # Replace with the module name of the package
    except ImportError:
        print(f"{'mplhep'} not found. Installing...")
        install_package("mplhep")
        import mplhep  # Import again after installation
    try:
        import hist  # Replace with the module name of the package
    except ImportError:
        print(f"{'hist'} not found. Installing...")
        install_package("hist")
        import hist  # Import again after installation
    try:
        import pickle  # Replace with the module name of the package
    except ImportError:
        print(f"{'pickle5'} not found. Installing...")
        install_package("pickle5")
        import pickle  # Import again after installation
        
import_packages()

In [3]:
mbinsDet = [   0.,    5.,   10.,   15.,   20.,   30.,   40.,   50.,   60.,
     70.,   80.,   90.,  100.,  125.,  150.,  175.,  200.,  250.,
    300.,  800., 1300.]

mbinsGen = [   0.,   10.,   20.,   40.,   60.,   80.,  100.,  150.,  200.,
    300., 1300.]


ptbinsGen = [200, 290, 400, 480, 570, 680,  13000]
ptbinsDet = ptbinsGen

nmbinsGen  = len(mbinsGen) -1
nmbinsDet  = len(mbinsDet) -1
nptbinsGen = len(ptbinsGen) -1
nptbinsDet = len(ptbinsDet) -1

In [4]:
generatorBinning = ROOT.TUnfoldBinning("generator")
### Need coarser binning for signal
signalBinning = generatorBinning.AddBinning("signal")    
signalBinning.AddAxis("massgen",nmbinsGen,mbinsGen,
                        False, # needed for fakes
                        False # overflow bin
                        )
signalBinning.AddAxis("ptgen",nptbinsGen,ptbinsGen,
                        False, # needed for fakes
                        False # overflow bin
                        )

# X axis : generator binning is Signal : mgen * ptgen and Background : mrec * ptrec

xaxis =  array.array('d',[a for a in range(nmbinsGen *nptbinsGen + 1 )])
nx = len(xaxis)-1

print (xaxis)
print("There are {} (signal) generator level bins in the simple scheme".format(nx))
print("---------------------------------------------------------------")

print("signalBinning")
for i in range(len(signalBinning.GetDistributionBinning(0))):
    print (signalBinning.GetDistributionBinning(0)[i])

#print "detectorBinning"
#for i in xrange(len(backgroundBinning.GetDistributionBinning(0))):
#    print backgroundBinning.GetDistributionBinning(0)[i]
    
#print "Below counts include overflow and underflow bins"    
#print "There are {} signal {} background bins in the generator scheme".format(signalBinning.GetTH1xNumberOfBins() , backgroundBinning.GetDistributionNumberOfBins())
genBin = generatorBinning

TypeError: none of the 3 overloaded methods succeeded. Full details:
  bool TUnfoldBinning::AddAxis(const TAxis& axis, bool includeUnderflow, bool includeOverflow) =>
    TypeError: takes at most 3 arguments (5 given)
  bool TUnfoldBinning::AddAxis(const char* name, int nBins, double xMin, double xMax, bool hasUnderflow, bool hasOverflow) =>
    TypeError: takes at least 6 arguments (5 given)
  bool TUnfoldBinning::AddAxis(const char* name, int nBins, const double* binBorders, bool hasUnderflow, bool hasOverflow) =>
    TypeError: could not convert argument 3 (could not convert argument to buffer or nullptr)

In [ ]:
detectorBinning = ROOT.TUnfoldBinning("detector") 
detectorDistribution=detectorBinning.AddBinning("detectordistribution")
detectorDistribution.AddAxis("mass",nmbinsDet,mbinsDet,
                        False, # no underflow bin (not reconstructed)
                        False # overflow bin
                        )
detectorDistribution.AddAxis("pt",nptbinsDet,ptbinsDet,
                        False, # no underflow bin (not reconstructed)
                        False # overflow bin
                        )

yaxis =  array.array('d',[a for a in range(nmbinsDet *nptbinsDet+1 )])
ny = len(yaxis)-1

print (yaxis)
print ("There are {}  detector level bins in the simple scheme".format(ny))
print ("detectorBinning")
for i in range(len(detectorDistribution.GetDistributionBinning(0))):
    print (detectorDistribution.GetDistributionBinning(0)[i])
    
print ("Below counts include overflow and underflow bins")
print ("There are {} bins in the detector scheme".format(detectorDistribution.GetTH1xNumberOfBins()))#GetDistributionNumberOfBins())
detBin  = detectorBinning

In [ ]:
import pickle as pkl
with open("qjetmass_zjets_gen__all_syst_v2.pkl", "rb") as f:
    output = pkl.load( f )
with open("qjetmass_zjets_reco_v2.pkl", "rb") as f:
    output_data = pkl.load( f )
    
    
with open("pkl_files/jackknife_output0to4_real.pkl", "rb") as f:
    output0to4 = pkl.load( f )
    
with open("pkl_files/jackknife_output_5to9.pkl", "rb") as f:
    output5to9 = pkl.load( f )

In [ ]:
with open("pt170-13000.pkl", 'rb') as f:
    output0to9 = pkl.load(f)

In [ ]:
ptreco_center = output['response_matrix_g'].axes[1].centers
ptreco_width = output['response_matrix_g'].axes[1].widths

ptgen_center = output['response_matrix_g'].axes[3].centers
ptgen_width = output['response_matrix_g'].axes[3].widths
mreco_center = output['response_matrix_g'].axes[2].centers
mreco_edge = output['response_matrix_g'].axes[2].edges
mgen_center = output['response_matrix_g'].axes[4].centers

mreco_width = output['response_matrix_g'].axes[2].widths
mgen_width = output['response_matrix_g'].axes[4].widths


In [ ]:
M = ROOT.TUnfoldBinning.CreateHistogramOfMigrations(generatorBinning,detectorBinning,"M")


h = detectorBinning.CreateHistogram("h")

hup = detectorBinning.CreateHistogram("hup")
hdn = detectorBinning.CreateHistogram("hdn")



htrue = generatorBinning.CreateHistogram("htrue")
htruef = detectorBinning.CreateHistogram("htruef")

In [ ]:
output0to4['jackknife_response_u'][{"jk": 0}]

In [ ]:
# M_np = output['tunfold_migration_u'][{'systematic':'nominal'}].project("bin_gen","bin_reco").values(flow = True)
# M_np_error = output['tunfold_migration_u'][{'systematic':'nominal'}].project("bin_gen","bin_reco").variances(flow = True)

# h_np = output_data['tunfold_reco_u'][{'systematic':'nominal'}].project("bin_reco").values(flow = True)
# h_np_error = output_data['tunfold_reco_u'][{'systematic':'nominal'}].project("bin_reco").variances(flow = True)

# htrue_np = output['tunfold_migration_u'][{'systematic':'nominal'}].project("bin_gen").values(flow = True)
# htrue_np_error = output['tunfold_migration_u'][{'systematic':'nominal'}].project("bin_gen").variances(flow = True)

# jk_dic = {}

# for i in range(10):
#     if (i<5):

#         Msyst = output0to4['jackknife_response_u'][{"jk": i}].project("bin_gen","bin_reco").view(flow=True).value
#         jk_dic.update({str(i):Msyst})

        
        
# M_np = output['tunfold_migration_g'][{'systematic':'nominal'}].project("bin_gen","bin_reco").values(flow = True)
# M_np_error = output['tunfold_migration_g'][{'systematic':'nominal'}].project("bin_gen","bin_reco").variances(flow = True)

# h_np = output_data['tunfold_reco_g'][{'systematic':'nominal'}].project("bin_reco").values(flow = True)
# h_np_error = output_data['tunfold_reco_g'][{'systematic':'nominal'}].project("bin_reco").variances(flow = True)

# htrue_np = output['tunfold_migration_g'][{'systematic':'nominal'}].project("bin_gen").values(flow = True)
# htrue_np_error = output['tunfold_migration_g'][{'systematic':'nominal'}].project("bin_gen").variances(flow = True)

# jk_dic = {}

# for i in range(10):
#     if (i<5):

#         Msyst = output0to4['jackknife_response_g'][{"jk": i}].project("bin_gen","bin_reco").view(flow=True).value
#         jk_dic.update({str(i):Msyst})
#     else:
#         Msyst = output5to9['jackknife_response_g'][{"jk": i-5}].project("bin_gen","bin_reco").view(flow=True).value
#         jk_dic.update({str(i):Msyst})

In [ ]:
closure = True
groomed = False

In [ ]:
if groomed == False:
    jk_dic = {}
    for i in range(10):
        print(i)
        Msyst = output0to9['jk_response_matrix_u'][{"jk": i}].project("mgen","ptgen","mreco","ptreco").view(flow=True).value
        jk_dic.update({str(i):Msyst})
        
    matrix_combined = np.stack(list(jk_dic.values()), axis = 0)
    rms = np.std(matrix_combined, axis = 0)
    rms = np.sqrt(10/9)*rms
    
    M_np = output['response_matrix_u'][{"systematic": "nominal"}].project("mgen","ptgen","mreco","ptreco").view(flow=True).value
    M_np_error = rms



    n_bin_det = M_np.shape[2]*M_np.shape[3]
    n_bin_gen =  M_np.shape[0]*M_np.shape[1]


    if closure:
        h_np = output['ptjet_mjet_u_reco'][{"systematic": "nominal"}].project("mreco","ptreco").view(flow=True).value
        h_np_error = output['ptjet_mjet_u_reco'][{"systematic": "nominal"}].project("mreco","ptreco").view(flow=True).variance
        h_np_error = np.sqrt(h_np_error)
    else:
        h_np = output['ptjet_mjet_u_reco'][{"systematic": "nominal"}].project("mreco","ptreco").view(flow=True).value
        h_np_error = output['ptjet_mjet_u_reco'][{"systematic": "nominal"}].project("mreco","ptreco").view(flow=True).variance
        h_np_error = np.sqrt(h_np_error)

    # htrue_np = np.sum(output['response_matrix_g'][{"systematic": "nominal"}].project("mgen","ptgen","mreco","ptreco").view(flow=True).value, (2,3))
    # htrue_np_error = np.sum(output['response_matrix_g'][{"systematic": "nominal"}].project("mgen","ptgen","mreco","ptreco").view(flow=True).variance, (2,3))
    # htrue_np_error = np.sqrt(htrue_np_error)

    htrue_np =output['response_matrix_u'][{"systematic": "nominal"}].project("mgen","ptgen").view(flow=True).value
    htrue_np_error = output['response_matrix_u'][{"systematic": "nominal"}].project("mgen","ptgen").view(flow=True).variance
    htrue_np_error = np.sqrt(htrue_np_error)

    

    


In [ ]:
if groomed:
    
    
    jk_dic = {}
    for i in range(10):
        print(i)
        Msyst = output0to9['jk_response_matrix_g'][{"jk": i}].project("mgen","ptgen","mreco","ptreco").view(flow=True).value
        jk_dic.update({str(i):Msyst})
        
    matrix_combined = np.stack(list(jk_dic.values()), axis = 0)
    rms = np.std(matrix_combined, axis = 0)
    rms = np.sqrt(10/9)*rms
    
    M_np = output['response_matrix_g'][{"systematic": "nominal"}].project("mgen","ptgen","mreco","ptreco").view(flow=True).value
    M_np_error = rms
    



    n_bin_det = M_np.shape[2]*M_np.shape[3]
    n_bin_gen =  M_np.shape[0]*M_np.shape[1]


    if closure:
        h_np = output['ptjet_mjet_g_reco'][{"systematic": "nominal"}].project("mreco","ptreco").view(flow=True).value
        h_np_error = output['ptjet_mjet_g_reco'][{"systematic": "nominal"}].project("mreco","ptreco").view(flow=True).variance
        h_np_error = np.sqrt(h_np_error)
    else:
        h_np = output['ptjet_mjet_g_reco'][{"systematic": "nominal"}].project("mreco","ptreco").view(flow=True).value
        h_np_error = output['ptjet_mjet_g_reco'][{"systematic": "nominal"}].project("mreco","ptreco").view(flow=True).variance
        h_np_error = np.sqrt(h_np_error)

    # htrue_np = np.sum(output['response_matrix_g'][{"systematic": "nominal"}].project("mgen","ptgen","mreco","ptreco").view(flow=True).value, (2,3))
    # htrue_np_error = np.sum(output['response_matrix_g'][{"systematic": "nominal"}].project("mgen","ptgen","mreco","ptreco").view(flow=True).variance, (2,3))
    # htrue_np_error = np.sqrt(htrue_np_error)

    htrue_np =output['response_matrix_g'][{"systematic": "nominal"}].project("mgen","ptgen").view(flow=True).value
    htrue_np_error = output['response_matrix_g'][{"systematic": "nominal"}].project("mgen","ptgen").view(flow=True).variance
    htrue_np_error = np.sqrt(htrue_np_error)



In [ ]:
output['response_matrix_u'][{"systematic": "nominal"}].project("mgen","ptgen","mreco","ptreco")

In [ ]:
jk_dic['0'].shape

In [ ]:
np.sum(output0to4['jackknife_response_u'][{"jk": 0}].project("bin_gen","bin_reco").values())

In [ ]:
np.sum(h_np)

In [ ]:
166689*(0.9)

In [ ]:
M.GetNbinsX()

In [ ]:
list_reco = []
list_gen = []
integral = 0
for i in range(M_np.shape[0]-2): #mgen
    for j in range(M_np.shape[1]-1): #ptgen
        for k in range(M_np.shape[2]-2): #mreco
            for l in range(M_np.shape[3]-1): #ptreco
                
                mgen = mgen_center[i]
                if j == 0:
                    ptgen = 1
                else:
                    ptgen = ptgen_center[j-1]
                mreco = mreco_center[k]
                if l == 0:
                    ptreco = 1
                else:
                    ptreco = ptreco_center[l-1]
    
                # glob_reco = detectorDistribution.GetGlobalBinNumber(mreco_center[k], ptreco_center[l])
                # glob_gen = signalBinning.GetGlobalBinNumber(mgen_center[i], ptgen_center[j])
                glob_reco = detectorDistribution.GetGlobalBinNumber(mreco, ptreco)
                glob_gen = signalBinning.GetGlobalBinNumber(mgen, ptgen)
                #rint(glob_gen, glob_reco)
                # list_reco.append(glob_reco)
                # list_gen.append(glob_gen)
                integral = integral + M_np[i+1][j][k+1][l]
                M.SetBinContent(glob_gen, glob_reco, M_np[i+1][j][k+1][l])
                M.SetBinError( glob_gen, glob_reco, M_np_error[i+1][j][k+1][l])
print(integral)                
jk_dic_root = {}                      
for i_jk in range(10):  
    jk_dic_root.update({str(i_jk): ROOT.TUnfoldBinning.CreateHistogramOfMigrations(generatorBinning,detectorBinning,str(i_jk))})
    for i in range(M_np.shape[0]-2): #mgen
        for j in range(M_np.shape[1]-1): #ptgen
            for k in range(M_np.shape[2]-2): #mreco
                for l in range(M_np.shape[3]-1): #ptreco
                    
                    mgen = mgen_center[i]
                    if j == 0:
                        ptgen = 1
                    else:
                        ptgen = ptgen_center[j-1]
                    mreco = mreco_center[k]
                    if l == 0:
                        ptreco = 1
                    else:
                        ptreco = ptreco_center[l-1]

                    glob_reco = detectorDistribution.GetGlobalBinNumber(mreco, ptreco)
                    glob_gen = signalBinning.GetGlobalBinNumber(mgen, ptgen)
                    jk_dic_root[str(i_jk)].SetBinContent(glob_gen, glob_reco, jk_dic[str(i_jk)][i+1,j,k+1,l])
                    #jk_dic_root[str(i_jk)].SetBinError(glob_gen, glob_reco, resp_syst_dic_np_error[sys][i+1,j,k+1,l])
                
for j in range(M_np.shape[1]-1): #ptgen                
    for i in range(M_np.shape[0]-2): #mgen  

        mgen = mgen_center[i]
        if j == 0:
            ptgen = 1
        else:
            ptgen = ptgen_center[j-1]
        
        glob_gen = signalBinning.GetGlobalBinNumber(mgen, ptgen)
        
        htrue.SetBinContent(glob_gen, htrue_np[i+1][j])
        htrue.SetBinError(glob_gen, htrue_np_error[i+1][j])
        
for l in range(M_np.shape[3]-1): #ptreco           
    for k in range(M_np.shape[2]-2): #mreco    
        mreco = mreco_center[k]
        if l == 0:
            ptreco = 1
        else:
            ptreco = ptreco_center[l-1]
        glob_reco = detectorDistribution.GetGlobalBinNumber(mreco, ptreco)
        #print(glob_reco)
        h.SetBinContent(glob_reco, h_np[k+1][l])
        h.SetBinError(glob_reco, h_np_error[k+1][l])

In [ ]:
ptgen_center_uf = np.insert(ptgen_center, 0, -5)
mgen_center_uf = np.insert(ptgen_center, 0, -5)
ptreco_center_uf = np.insert(ptreco_center, 0, -5)
mgen_width_uf = np.insert(mgen_width, 0, 1)
mgen_width_f = np.insert(mgen_width_uf, -1, 10000)
mreco_width_uf = np.insert(mreco_width, 0, 1)


def NormM(M, mode = 'gen'):
    if mode == 'gen':
        print("Normalising in GEN axis")
        M_norm = M.Clone("M_norm")
        for pt in ptgen_center_uf:
            start_gen = signalBinning.GetGlobalBinNumber(-1,pt)
            end_gen = signalBinning.GetGlobalBinNumber(mgen_center_uf[-1]+10000,pt)
            # print(start_gen)
            # print(end_gen)
            # print(end_gen-start_gen)
            binsum = M.Integral(start_gen, end_gen, 0, M.GetNbinsY()+1)
            print(binsum)
            im = 0
            for ix in range(start_gen, end_gen+1):
                for iy in range(0, M.GetNbinsY()+1):
                    if binsum>0:
                        M_norm.SetBinContent(ix, iy, M.GetBinContent(ix, iy)/(binsum*mgen_width_f[im]) )
                im+=1
        return M_norm
    if mode == 'reco':
        print("Normalising in RECO axis")
        M_norm = M.Clone("M_norm")
        for pt in ptreco_center_uf:
            start_reco = detectorDistribution.GetGlobalBinNumber(-1,pt)
            end_reco = detectorDistribution.GetGlobalBinNumber(mgen_center_uf[-1]+10000,pt)
            # print(start_reco)
            # print(end_reco)
            # print(end_reco-start_reco)
            binsum = M.Integral( 0, M.GetNbinsX()+1, start_reco, end_reco)
            print(binsum)
            im = 0
            for ix in range(0, M.GetNbinsX()+1):
                for iy in range(start_reco, end_reco+1):
                    if binsum>0:
                        M_norm.SetBinContent(ix, iy, M.GetBinContent(ix, iy)/(binsum) )
                im+=1
        return M_norm

def NormOutput(h):
    h_norm = h.Clone("M_norm")
    for pt in ptgen_center_uf:
        start_gen = signalBinning.GetGlobalBinNumber(-1,pt)
        end_gen = signalBinning.GetGlobalBinNumber(mgen_center[-1]+10000,pt)
        
        binsum = h.Integral(start_gen, end_gen)
        print(binsum)
        im = 0
        for ix in range(start_gen, end_gen+1):
            if binsum>0:
                h_norm.SetBinContent(ix,  h.GetBinContent(ix)/(binsum*mgen_width_f[im]) )
            im+=1
    return h_norm

def NormInput(h):
    h_norm = h.Clone("M_norm")
    for pt in ptreco_center_uf:
        start_reco = detectorDistribution.GetGlobalBinNumber(-1,pt)
        end_reco = detectorDistribution.GetGlobalBinNumber(mreco_center[-1]+10000,pt)
        binsum = h.Integral(start_reco, end_reco)
        print(binsum)
        im = 0
        for ix in range(start_reco, end_reco):
            if binsum>0:
                h_norm.SetBinContent(ix,  h.GetBinContent(ix)/(binsum*mreco_width_uf[im]) )
            im+=1
    return h_norm

norm = True
if norm:
    mode = 'gen'
    M = NormM(M, mode = mode)
    htrue = NormOutput(htrue)
    h = NormInput(h)
    for key in jk_dic_root.keys():
        jk_dic_root[key] = NormM(jk_dic_root[key], mode = mode )

In [ ]:
orientation = ROOT.TUnfold.kHistMapOutputHoriz
regMode = ROOT.TUnfold.kRegModeCurvature
con = ROOT.TUnfold.kEConstraintArea #ROOT.TUnfold.kEConstraintArea
mode =  ROOT.TUnfoldDensity.kDensityModeBinWidth
axisSteering =  "*[UOB]"

#u = ROOT.TUnfoldDensity(M, orientation, regMode, con, mode, genBin, detBin, "signal", axisSteering)
u_list = []
for i in range(10):
    u_list.append(ROOT.TUnfoldDensity(jk_dic_root[str(i)],orientation,regMode,con))

    u_list[i].SetInput(h)


    nScan=50
    tauMin=0.0
    tauMax=0.0
    iBest=0

    logTauX = ROOT.MakeNullPointer(ROOT.TSpline)
    logTauY = ROOT.MakeNullPointer(ROOT.TSpline)
    lCurve = ROOT.MakeNullPointer(ROOT.TGraph)
    u_list[i].ScanLcurve(nScan,tauMin,tauMax,lCurve,logTauX,logTauY)
    
    

In [ ]:
o_list = []
for i in range(10):
    o_list.append( u_list[i].GetOutput("o"+str(i)))

In [ ]:
a = u_list[0].GetEmatrixSysUncorr('mat')

In [ ]:
o_unc = o_list[0].Clone('unc')
o_unc.Reset()
for i in range(1, o_unc.GetNbinsX()):
    o_unc.SetBinContent(i, o_list[0].GetBinContent(i))
    o_unc.SetBinError(i, a.GetBinContent(i,i)**0.5)

In [ ]:
c_output = ROOT.TCanvas()
for i in range(10):
    o_list[i].SetLineColor(i)
    o_list[i].Draw("hist same")
    
c_output.Draw()

In [ ]:
%jsroot

In [ ]:
c_new = ROOT.TCanvas()
o_unc.SetLineWidth(2)
o_unc.SetFillColor(2)
o_unc.SetFillStyle(2)
o_unc.Draw('e')
c_new.Draw()

In [ ]:
bin_avg = []
bin_rms = []
for i_bin in range(o_list[0].GetNbinsX()):
    content_list = []
    for i_jk in range(10):
        content_list.append(o_list[i_jk].GetBinContent(i_bin))
    bin_avg.append(np.mean(content_list) )
    mean_content = np.mean(content_list)
    std = np.std(content_list)
    error = np.sqrt(10/9)*std
    bin_rms.append(error)
        

In [ ]:
plt.plot(h_np/np.sum(h_np, axis = 0))

In [ ]:
np.sum(h_np, axis = 0)

In [ ]:
bin_rms

In [ ]:
bin_rms = np.array(bin_rms)
if not groomed:
    np.savetxt('rms_data_u.txt', bin_rms)
else:
    np.savetxt('rms_data_g.txt', bin_rms)

In [ ]:
plt.plot(bin_avg)

In [ ]:
plt.plot(  bin_rms/np.array(bin_avg), marker = 'o')
plt.xlabel('Bin number')
plt.ylabel('Bin Error')

In [ ]:
import numpy as np

sample_size = 1000000
data = np.random.normal(loc=0, scale=1, size=sample_size)  

sample_mean = np.mean(data)
sample_std = np.std(data, ddof=1)
standard_error_rms = sample_std / np.sqrt(sample_size)

# Step 3: Calculate Standard Error using 10-fold Jackknife Resampling
k = 10
n = len(data)
subset_size = n // k
jackknife_means = []

for i in range(k):
    start_index = i * subset_size
    end_index = start_index + subset_size if i < k - 1 else n
    jackknife_sample = np.delete(data, np.arange(start_index, end_index))
    jackknife_mean = np.mean(jackknife_sample)
    jackknife_means.append(jackknife_mean)

jackknife_means = np.array(jackknife_means)
mean_jackknife_means = np.mean(jackknife_means)
jackknife_variance = (k - 1) / k * np.sum((jackknife_means - mean_jackknife_means) ** 2)
standard_error_jackknife = np.sqrt(jackknife_variance)

# Step 4: Compare the Results
print("Standard Error using RMS method:", standard_error_rms)
print("Standard Error using 10-fold Jackknife method:", standard_error_jackknife)

In [ ]:
jackknife_means

In [ ]:
signalBinning.GetGlobalBinNumber(5,261)

In [ ]:
plt.errorbar( mgen_center, bin_avg[23: 32], bin_rms[23: 32], marker = '.')
plt.xlim(0,200)

In [ ]:
plt.errorbar( mgen_center, bin_avg[34: 43], bin_rms[34: 43], marker = 'o')
plt.xlim(0,200)

In [ ]:
canvs = []
hists = []
hcopy_list = []
i = 2
for ipt in range(i,i+1): 
    ci = ROOT.TCanvas("c" + str(ipt), "c" + str(ipt))
    
    for i_jk in range(10):
        hcopy_list.append(ROOT.TH1F("unfolded_pt" + str(ipt)+str(i_jk), "Unfolded Ungroomed Jet Mass for p_{T} " + "  "+  str(i_jk)+ str(ptbinsGen[ipt+1])[:-2] +" - " +str(ptbinsGen[ipt+2])[:-2] +' GeV', nmbinsGen, mbinsGen ))
        m_int = 0
        for im in range(nmbinsGen):
            #print(im + ipt*(nmbinsGen+2) + 2)
            # print("mass ", mgen_center[im])
            # print("pt " , ptgen_center[ipt])

            bin_number = signalBinning.GetGlobalBinNumber(mgen_center[im], ptgen_center[ipt])

            m_int = m_int + o_list[i].GetBinContent(bin_number)
        for im in range(nmbinsGen):
            #print(im + ipt*(nmbinsGen+2) + 2)
            # print("mass ", mgen_center[im])
            # print("pt " , ptgen_center[ipt])

            bin_number = signalBinning.GetGlobalBinNumber(mgen_center[im], ptgen_center[ipt])
            if (m_int>0):
                hcopy_list[i_jk].SetBinContent(im+1, o_list[i_jk].GetBinContent(bin_number )/(mgen_width[im]*m_int) )
        hcopy_list[i_jk].SetLineColor(i_jk)
        hcopy_list[i_jk].GetXaxis().SetRangeUser(0,200)
        hcopy_list[i_jk].Draw('hist same')
    ci.Draw()








    

In [ ]:
%jsroot

In [ ]:
c = ROOT.TCanvas()
for h in hcopy_list:
    h.Draw('hist same')
c.Draw()   

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Generate example data (truth and measured distributions)
np.random.seed(0)  # For reproducibility

# Known truth distribution
pt_bins = np.array([0, 10, 20, 30, 40, 50])
mass_bins = np.linspace(0, 10, 51)  # Mass bins for each pt bin

# Example truth distributions for each pt-bin
truth_distributions = [np.random.poisson(lam=(i+1)*100, size=len(mass_bins)-1) for i in range(len(pt_bins)-1)]

# Normalize each pt-bin for truth distributions
normalized_truth_distributions = [truth / truth.sum() for truth in truth_distributions]

# Simulate measured data with some noise
measured_distributions = [np.random.poisson(lam=(i+1)*100 + np.random.normal(0, 10), size=len(mass_bins)-1) for i in range(len(pt_bins)-1)]

# Normalize each pt-bin for measured distributions
normalized_measured_distributions = [measured / measured.sum() for measured in measured_distributions]

# Combine normalized data into single arrays for 2D histogram
mass_data_truth = np.concatenate([mass_bins[:-1] for _ in range(len(pt_bins)-1)])
mass_data_measured = np.concatenate([mass_bins[:-1] for _ in range(len(pt_bins)-1)])
truth_data = np.concatenate([np.repeat(pt_bins[i], len(mass_bins)-1) for i in range(len(pt_bins)-1)])
measured_data = np.concatenate([np.repeat(pt_bins[i], len(mass_bins)-1) for i in range(len(pt_bins)-1)])

# Step 2: Create and fill the 2D histogram (response matrix)
response_matrix, xedges, yedges = np.histogram2d(
    np.concatenate([mass_data_truth for _ in range(len(pt_bins)-1)]),
    np.concatenate([mass_data_measured for _ in range(len(pt_bins)-1)]),
    bins=(mass_bins, mass_bins)
)

# Normalize the response matrix (row-wise)
row_sums = response_matrix.sum(axis=1, keepdims=True)
normalized_response_matrix = response_matrix / row_sums

# Verify normalization
print("Normalized Response Matrix Row Sums:", normalized_response_matrix.sum(axis=1))  # Should print an array of ones

# Step 3: Fill 1D histograms for truth and measured distributions
truth_hist, _ = np.histogram(np.concatenate([mass_data_truth for _ in range(len(pt_bins)-1)]), bins=mass_bins)
measured_hist, _ = np.histogram(np.concatenate([mass_data_measured for _ in range(len(pt_bins)-1)]), bins=mass_bins)

# Step 4: Unfold the measured distribution
pseudo_inverse_response_matrix = np.linalg.pinv(normalized_response_matrix)
unfolded_hist = pseudo_inverse_response_matrix.dot(measured_hist)

# Step 5: Denormalize the unfolded distribution
denormalized_unfolded_hist = unfolded_hist * measured_hist.sum() / unfolded_hist.sum()

# Plot the distributions for comparison
fig, ax = plt.subplots(figsize=(10, 6))

width = 0.25  # Width of the bars

# Plot the original truth distributions
for i, truth in enumerate(truth_distributions):
    ax.bar(mass_bins[:-1], truth, width, label=f'Original Truth pt-bin {pt_bins[i]}-{pt_bins[i+1]}', alpha=0.5)

# Plot the measured distributions
for i, measured in enumerate(measured_distributions):
    ax.bar(mass_bins[:-1] + width, measured, width, label=f'Measured pt-bin {pt_bins[i]}-{pt_bins[i+1]}', alpha=0.5)

# Plot the unfolded distributions
ax.bar(mass_bins[:-1] + 2 * width, denormalized_unfolded_hist, width, label=f'Unfolded', alpha=0.5)

ax.set_xlabel('Mass Bins')
ax.set_ylabel('Counts')
ax.set_title('Unfolding with Normalized pt-bins')
ax.legend()

plt.show()

# Print the distributions for clarity
print("Original Truth Distribution:", truth_distributions)
print("Measured Distribution:", measured_distributions)
print("Unfolded Distribution:", denormalized_unfolded_hist)


In [ ]:
plt.plot(truth_hist)